# Making new layers and models via subclassing
### Juntado todo: Ejemplo de principio a fin

Esto es lo que se ha aprendido:
* Una *layer* encapsula un estado (creado en *__init__()* o en *build()*) y algunos cómputos (definidos en *call()*).
* Las capas pueden anidarse recursivamente para crear nuevos bloques de cálculo más grandes.
* Las capas son agnósticas de backend siempre y cuando solo utilicen APIs de Keras. Se puede utilizar APIs nativas de backed (como jax.numpy, torch.nn o tf.nn), pero entonces la capa será solo utilizable con ese backend específico.
* Las capas pueden crear y rastrear pérdidas (típicamente pérdidas de regularación) a través de *add_loss()*.
* El contenedor externo, es decir, lo que se desea entrenar, es un Modelo. Un *Model* es como una *Layer*, pero con utilidades añadidas de entrenamiento y serialización.

**Se implementará un Variational AutoEncoder (VAE) en un *backed-agnostic fashion*, de forma que corra o mismo en Tensorflow, JAX, y PyTorch. Se entreanará con dígitos MNIST.**

El VAE que se creará será una subclase de *Model*, construído como una composición de cpas anidades de la subclase *Layer*. Tendrá una pérdida de regularización (KL divergence).

In [ ]:
import keras
import numpy as np
from keras import ops

In [ ]:
# Capa personalizada de Keras que realiza el muestre de una distribución latente.
class Sampling(keras.layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    # Inicializa la capa y  configura un generador de semillas para la aleatoriedad
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.seed_generator = keras.random.SeedGenerator(1337)

    # Método que usa en el paso forward del modelo.
    # Toma como entrada 'z_mean' y 'z_log_var' que son la media
    # y el logaritmo de la varianza de una distribución normal.
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = ops.shape(z_mean)[0]
        dim = ops.shape(z_mean)[1]
        epsilon = keras.random.normal(shape=(batch, dim), seed=self.seed_generator)
        return z_mean + ops.exp(0.5 * z_log_var) * epsilon
        # Muestrea la distribución nromal mediante la técnica de 'reparametrización'
        # que es crucial para que el modelo pueda entrenarse mediante backpropagation.


# Define una capa que mapea las entradas (MNIS) a un trío de vector:
# 'z_mean', 'z_log_Var', y 'z'
class Encoder(keras.layers.Layer):
    """Maps MNIST digits to a triplet (z_mean, z_log_var, z)."""

    # Constructor que inicializa la capa y establece capas densas para proyeccciones
    # intermedias, cálculo de media y log-varianza.
    def __init__(self, latent_dim=32, intermediate_dim=64, name="encoder", **kwargs):
        super().__init__(name=name, **kwargs)
        self.dense_proj = keras.layers.Dense(intermediate_dim, activation="relu")
        self.dense_mean = keras.layers.Dense(latent_dim)
        self.dense_log_var = keras.layers.Dense(latent_dim)
        self.sampling = Sampling()

  # Procesa las entradas a través de las capas densas y utiliza la clase 'Sampling'
  # para obtener el vector latente 'z'
    def call(self, inputs):
        x = self.dense_proj(inputs)
        z_mean = self.dense_mean(x)
        z_log_var = self.dense_log_var(x)
        z = self.sampling((z_mean, z_log_var))
        return z_mean, z_log_var, z


# Invierte el proceso del encoder, transformando el vector latente 'z' de nuevo
# a una representación legible (un dígito)
class Decoder(keras.layers.Layer):
    """Converts z, the encoded digit vector, back into a readable digit."""

    # Constructor que inicializa la capa con capas densas
    def __init__(self, original_dim, intermediate_dim=64, name="decoder", **kwargs):
        super().__init__(name=name, **kwargs)
        self.dense_proj = keras.layers.Dense(intermediate_dim, activation="relu")
        self.dense_output = keras.layers.Dense(original_dim, activation="sigmoid")

    # Procesa el vector latente a través de las capas densas para reconstruir lla salida
    def call(self, inputs):
        x = self.dense_proj(inputs)
        return self.dense_output(x)


# Combina el 'Encoder' y 'Decoder' en un modelo integral para entrenamiento y
# reconstrucción
class VariationalAutoEncoder(keras.Model):
    """Combines the encoder and decoder into an end-to-end model for training."""

    # Constructor que inicializa el VAE, estableciendo las dimensiones y creando las
    # las isntancias de 'Encoder' y 'Decoder
    def __init__(
        self,
        original_dim,
        intermediate_dim=64,
        latent_dim=32,
        name="autoencoder",
        **kwargs
    ):
        super().__init__(name=name, **kwargs)
        self.original_dim = original_dim
        self.encoder = Encoder(latent_dim=latent_dim, intermediate_dim=intermediate_dim)
        self.decoder = Decoder(original_dim, intermediate_dim=intermediate_dim)

    # Define el paso forward del modelo completo. Toma las entradas, las pasa através
    # del 'Encoder' para obtener 'z', y luego atavés del 'Decoder' para reconstruir
    # las entradas. Calcula la pérdida de divergencia KL, que es un componente
    # escencial de los VAEs, y las agrega a las pérdidas del modelo.
    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        # Add KL divergence regularization loss.
        kl_loss = -0.5 * ops.mean(
            z_log_var - ops.square(z_mean) - ops.exp(z_log_var) + 1
        )
        self.add_loss(kl_loss)
        return reconstructed

In [ ]:
(x_train, _), _ = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000,784).astype('float32')/255

original_dim = 784
vae = VariationalAutoEncoder(784, 64, 32)

optimizer = keras.optimizers.Adam(learning_rate = 1e-3)
vae.compile(optimizer, loss=keras.losses.MeanSquaredError())

vae.fit(x_train, x_train, epochs=2, batch_size=64)

Epoch 1/2
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.0938
Epoch 2/2
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 0.0677
